### **Installing Transformers and Importing Dependencies.**

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 901kB 22.7MB/s 
     |████████████████████████████████| 3.3MB 33.3MB/s 


In [2]:
!pip install sentence-splitter

     |████████████████████████████████| 51kB 2.6MB/s 


In [3]:
!pip install SentencePiece

     |████████████████████████████████| 1.2MB 4.2MB/s 


In [4]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [56]:
## Paraphrasing one sentence!

text = "Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence"

In [57]:
## Checking Three responses!

get_response(text, 3)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['Artificial intelligence is a branch of computer science that builds machines that are capable of performing certain tasks.',
 'Artificial intelligence is a branch of computer science that builds machines that are capable of performing tasks that require human intelligence.',
 'Artificial intelligence is a branch of computer science concerned with building machines that are capable of performing certain tasks.']

In [58]:
## Getting One response!!
get_response(text, 1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['Artificial intelligence is a branch of computer science that builds machines that are capable of performing certain tasks.']

#**Loading Necessary libraries!!!**

In [7]:

from transformers import pipeline

In [8]:
## Initializing Summarizer!!

summarizer = pipeline('summarization')

In [40]:
ARTICLE = '''AB-Flo-N Tablet is taken with or without food in a dose and duration as advised by the doctor. The dose you are given will depend on your condition and how you respond to the medicine. You should keep taking this medicine for as long as your doctor recommends. If you stop treatment too early your symptoms may come back and your condition may worsen. Let your healthcare team know about all other medications you are taking as some may affect, or be affected by this medicine. The most common side effects are vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation. Most of these are temporary and usually resolve with time. Contact your doctor straight away if you are at all concerned about any of these side effects. Avoid drinking alcohol while taking this medicine as that may cause excessive drowsiness. Never support self-medication or recommend your medicine to another person. Before you start taking this medicine it is important to inform your doctor if you are suffering from liver or kidney disease. You should also tell your doctor if you are pregnant, planning pregnancy, or breastfeeding.'''

## Summary

In [41]:
summary_text = summarizer(ARTICLE, max_length=150, min_length=60)[0]['summary_text']

In [42]:
print("Summary:", summary_text)

Summary:  AB-Flo-N Tablet is taken with or without food in a dose and duration as advised by the doctor . Most common side effects are vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation . Avoid drinking alcohol while taking this medicine as that may cause excessive drowsiness .


In [50]:
# Takes the input paragraph and splits it into a list of sentences bcz it can parapharse single sentence at a time!
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(summary_text)
sentence_list

['AB-Flo-N Tablet is taken with or without food in a dose and duration as advised by the doctor .',
 'Most common side effects are vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation .',
 'Avoid drinking alcohol while taking this medicine as that may cause excessive drowsiness .']

In [51]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


In [52]:
paraphrase


[["The doctor's advice is to take the tablet with or without food."],
 ['Vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation are some of the common side effects.'],
 ["It's a good idea to avoid drinking alcohol while taking this medicine."]]

In [53]:
## converting list of list in a single list.

paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

["The doctor's advice is to take the tablet with or without food.",
 'Vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation are some of the common side effects.',
 "It's a good idea to avoid drinking alcohol while taking this medicine."]

In [54]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'"The doctor\'s advice is to take the tablet with or without food. Vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation are some of the common side effects. It\'s a good idea to avoid drinking alcohol while taking this medicine."'

In [55]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(summary_text)
print("*********************************************************")
print(paraphrased_text)

 AB-Flo-N Tablet is taken with or without food in a dose and duration as advised by the doctor . Most common side effects are vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation . Avoid drinking alcohol while taking this medicine as that may cause excessive drowsiness .
*********************************************************
"The doctor's advice is to take the tablet with or without food. Vomiting, heartburn, stomach upset, rash, itching, breathing difficulty, and nasal inflammation are some of the common side effects. It's a good idea to avoid drinking alcohol while taking this medicine."
